In [ ]:
import pandas as pd
import pandas_datareader.data as fred
from pandas_datareader import wb
import datetime as dt

# ---------- Helpers ----------
def to_quarter_avg(s):
    s = pd.Series(s).dropna()
    s.index = pd.to_datetime(s.index)
    return s.resample("Q").mean()

def to_quarter_ffill(s):
    s = pd.Series(s).dropna()
    s.index = pd.to_datetime(s.index)
    return s.resample("Q").ffill()

def wb_annual_to_quarter(s):
    s = pd.Series(s).dropna()
    s.index = pd.to_datetime([f"{int(y)}-12-31" for y in s.index])
    return s.resample("Q").ffill()

# ---------- Rango ----------
START = dt.datetime(2014,1,1)
END   = dt.datetime(2024,12,31)

# ---------- FRED / World Bank series ----------
# 1) DXY
dxy = fred.DataReader("DTWEXBGS", "fred", START, END).squeeze().rename("usd_index")
dxy_q = to_quarter_avg(dxy)

# 2) Tasa real USA (TIPS 10y)
real_rate = fred.DataReader("DFII10", "fred", START, END).squeeze().rename("us_real_rate")
real_rate_q = to_quarter_avg(real_rate)

# 3) Inflación USA (IPC YoY %)
cpi = fred.DataReader("CPIAUCSL", "fred", START, END).squeeze()
us_inflation = (cpi.pct_change(12) * 100).rename("us_inflation")
us_inflation_q = to_quarter_avg(us_inflation)

# 4) Petróleo Brent
oil = fred.DataReader("DCOILBRENTEU", "fred", START, END).squeeze().rename("oil_price_usd_bbl")
oil = pd.to_numeric(oil, errors="coerce")
oil_q = to_quarter_avg(oil)

# 5) S&P 500
spx = fred.DataReader("SP500", "fred", START, END).squeeze().rename("sp500_index")
spx_q = to_quarter_avg(spx)

# 6) VIX
vix = fred.DataReader("VIXCLS", "fred", START, END).squeeze().rename("vix_index")
vix_q = to_quarter_avg(vix)

# 7) PIB China (World Bank anual % crecimiento)
chn = wb.download(indicator="NY.GDP.MKTP.KD.ZG", country="CHN", start=2014, end=2024)
china_gdp_growth_annual = chn.droplevel(0)["NY.GDP.MKTP.KD.ZG"].rename("china_gdp_growth")
china_gdp_growth_q = wb_annual_to_quarter(china_gdp_growth_annual)

# 8) Deuda/PIB USA
debt_gdp = fred.DataReader("GFDEGDQ188S", "fred", START, END).squeeze().rename("us_gov_debt_gdp")
debt_gdp_q = to_quarter_ffill(debt_gdp)


# ---------- Unir todo ----------
macro_q = pd.concat([
    dxy_q, real_rate_q, us_inflation_q,
    oil_q, spx_q, vix_q, china_gdp_growth_q,
    debt_gdp_q
], axis=1).sort_index()

macro_q.index.name = "date"
macro_q = macro_q.reset_index()

# Filtrar rango Q1-2014 … Q4-2024
macro_q = macro_q[(macro_q["date"] >= "2014-03-31") & (macro_q["date"] <= "2024-12-31")]

# ---------- Exportar ----------
macro_q.to_excel("macro_quarterly_2014_2024.xlsx", index=False)
print("✅ Archivo generado: macro_quarterly_2014_2024.xlsx")
print(macro_q.head())